# PRABU - Incremental Learning untuk Prediksi Risiko Kredit Lintas Sektor

Notebook ini mendemonstrasikan penggunaan model Machine Learning dengan kemampuan pembelajaran inkremental untuk memprediksi risiko kredit. Model ini dirancang untuk dapat belajar dari data historis dan data dari berbagai sektor industri.

## 1. Setup dan Import Library

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib

# Tambahkan path ke direktori root proyek jika perlu (misalnya, jika notebook ada di subfolder)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from PrabuModule import incremental_model_trainer
from PrabuModule import ml_credit_risk_predictor

# Reload modul untuk memastikan perubahan terbaru terbaca (berguna saat development)
importlib.reload(incremental_model_trainer)
importlib.reload(ml_credit_risk_predictor)

DATASET_DIR = os.path.join(module_path, 'PrabuModule', 'datasets')

# Hapus model dan preprocessor lama untuk demo yang bersih setiap kali dijalankan
print("Menghapus model, preprocessor, dan file kelas lama (jika ada) untuk demo...")
if os.path.exists(incremental_model_trainer.MODEL_PATH): 
    os.remove(incremental_model_trainer.MODEL_PATH)
if os.path.exists(incremental_model_trainer.PREPROCESSOR_PATH): 
    os.remove(incremental_model_trainer.PREPROCESSOR_PATH)
if os.path.exists(incremental_model_trainer.CLASSES_PATH): 
    os.remove(incremental_model_trainer.CLASSES_PATH)
print("File-file lama berhasil dihapus.")

## 2. Memuat Semua Dataset Sintetis
Data dari berbagai sektor dan perusahaan dimuat untuk simulasi.

In [ ]:
all_data = incremental_model_trainer.load_all_datasets(dataset_folder=DATASET_DIR)
if not all_data.empty:
    print(f"Total data yang dimuat dari semua file CSV: {len(all_data)} baris")
    print(f"Kolom yang ada: {all_data.columns.tolist()}")
    print("\nContoh beberapa baris data:")
    display(all_data.head())
    print("\nDistribusi Kategori Risiko (TARGET_COLUMN):")
    display(all_data[incremental_model_trainer.TARGET_COLUMN].value_counts())
    print("\nDistribusi Sektor:")
    display(all_data['Sektor'].value_counts())
else:
    print("Tidak ada data yang dimuat. Pastikan file CSV ada di direktori yang benar.")

## 3. Pelatihan Model Awal
Model dilatih pada sebagian data awal. Misalnya, data dari sektor 'Pertambangan' dan 'Konstruksi' hingga tahun 2021.

In [ ]:
current_model = None
current_preprocessor = None

if not all_data.empty:
    # Pilih data untuk pelatihan awal
    # Misal, ambil data dari 2 sektor pertama dan hingga tahun 2021
    initial_sectors = ['Pertambangan', 'Konstruksi']
    df_initial_train = all_data[
        (all_data['Sektor'].isin(initial_sectors)) &
        (all_data['PeriodeTahun'] <= 2021)
    ].copy()

    if not df_initial_train.empty:
        print(f"Jumlah data untuk pelatihan awal: {len(df_initial_train)} baris")
        print(f"Sektor dalam data awal: {df_initial_train['Sektor'].unique().tolist()}")
        print(f"Tahun dalam data awal: {df_initial_train['PeriodeTahun'].unique().tolist()}")
        
        current_model, current_preprocessor = incremental_model_trainer.train_initial_model(df_initial_train, force_retrain_preprocessor=True)
        if current_model and current_preprocessor:
            print("\nModel awal berhasil dilatih.")
        else:
            print("\nGagal melatih model awal.")
    else:
        print("Tidak ada data yang memenuhi kriteria untuk pelatihan awal.")
else:
    print("Tidak ada data untuk diproses.")

## 4. Pembelajaran Inkremental: Menambahkan Data dari Sektor Baru
Simulasikan kedatangan data dari sektor baru (misalnya, 'Agro') dan perbarui model.

In [ ]:
if current_model and current_preprocessor and not all_data.empty:
    df_new_sector_data = all_data[
        (all_data['Sektor'] == 'Agro') &
        (all_data['PeriodeTahun'] <= 2021) # Ambil data historis awal sektor baru juga
    ].copy()

    if not df_new_sector_data.empty:
        print(f"\n--- Memperbarui model dengan data sektor baru (Agro) --- ({len(df_new_sector_data)} baris)")
        current_model = incremental_model_trainer.update_model_incrementally(
            df_new_sector_data, 
            model=current_model, 
            preprocessor=current_preprocessor
        )
        if current_model:
            print("Model berhasil diperbarui dengan data sektor Agro.")
    else:
        print("\nTidak ada data untuk sektor Agro yang memenuhi kriteria.")
else:
    print("Model awal belum dilatih atau tidak ada data, langkah ini dilewati.")

## 5. Pembelajaran Inkremental: Menambahkan Data Historis Baru
Simulasikan kedatangan data historis baru (misalnya, tahun 2022 dan 2023) untuk sektor yang sudah ada.

In [ ]:
if current_model and current_preprocessor and not all_data.empty:
    # Ambil data dari tahun 2022 dan 2023 untuk semua sektor yang sudah dikenal model
    # (Pertambangan, Konstruksi, Agro pada titik ini)
    known_sectors = ['Pertambangan', 'Konstruksi', 'Agro'] 
    df_new_historical_data = all_data[
        (all_data['Sektor'].isin(known_sectors)) &
        (all_data['PeriodeTahun'] > 2021) 
    ].copy()

    if not df_new_historical_data.empty:
        print(f"\n--- Memperbarui model dengan data historis baru (Tahun > 2021) --- ({len(df_new_historical_data)} baris)")
        print(f"Sektor dalam data historis baru: {df_new_historical_data['Sektor'].unique().tolist()}")
        print(f"Tahun dalam data historis baru: {df_new_historical_data['PeriodeTahun'].unique().tolist()}")
        current_model = incremental_model_trainer.update_model_incrementally(
            df_new_historical_data, 
            model=current_model, 
            preprocessor=current_preprocessor
        )
        if current_model:
            print("Model berhasil diperbarui dengan data historis baru.")
    else:
        print("\nTidak ada data historis baru yang memenuhi kriteria.")
else:
    print("Model awal belum dilatih atau tidak ada data, langkah ini dilewati.")

## 6. Pembelajaran Inkremental: Menambahkan Sektor Lainnya
Tambahkan sisa sektor (Manufaktur Alat Berat, Logistik Alat Berat) untuk semua periode waktu.

In [ ]:
if current_model and current_preprocessor and not all_data.empty:
    remaining_sectors = ['Manufaktur Alat Berat', 'Logistik Alat Berat']
    df_remaining_sector_data = all_data[all_data['Sektor'].isin(remaining_sectors)].copy()

    if not df_remaining_sector_data.empty:
        print(f"\n--- Memperbarui model dengan data sektor sisanya --- ({len(df_remaining_sector_data)} baris)")
        print(f"Sektor dalam data baru: {df_remaining_sector_data['Sektor'].unique().tolist()}")
        current_model = incremental_model_trainer.update_model_incrementally(
            df_remaining_sector_data, 
            model=current_model, 
            preprocessor=current_preprocessor
        )
        if current_model:
            print("Model berhasil diperbarui dengan data sektor sisanya.")
    else:
        print("\nTidak ada data untuk sektor sisanya.")
else:
    print("Model awal belum dilatih atau tidak ada data, langkah ini dilewati.")

## 7. Demonstrasi Prediksi
Lakukan prediksi pada beberapa sampel data menggunakan model yang sudah final (setelah semua pembaruan).

In [ ]:
if not all_data.empty:
    # Reload model dan preprocessor terbaru dari file (untuk memastikan kita menggunakan versi yang disimpan)
    # ml_credit_risk_predictor._load_resources() # Panggil ini jika ingin paksa reload di modulnya
    # atau kita bisa memuatnya secara manual di sini untuk pengujian notebook
    final_model = None
    final_preprocessor = None
    if os.path.exists(incremental_model_trainer.MODEL_PATH):
        final_model = joblib.load(incremental_model_trainer.MODEL_PATH)
    if os.path.exists(incremental_model_trainer.PREPROCESSOR_PATH):
        final_preprocessor = joblib.load(incremental_model_trainer.PREPROCESSOR_PATH)

    if final_model and final_preprocessor:
        print("\n--- Melakukan prediksi dengan model final ---")
        # Ambil 1 sampel acak dari setiap sektor untuk prediksi
        sample_predictions = []
        for sector in all_data['Sektor'].unique():
            sample_from_sector = all_data[all_data['Sektor'] == sector].sample(1, random_state=42)
            if not sample_from_sector.empty:
                sample_input_series = sample_from_sector.iloc[0]
                # Hapus target asli & kolom non-fitur dari sampel input sebelum prediksi
                features_to_drop = [incremental_model_trainer.TARGET_COLUMN, 'UniqueID', 'NamaPerusahaan', 'PeriodeTahun', 'PeriodeKuartal']
                sample_input_dict_full = sample_input_series.to_dict()
                sample_input_dict_for_pred = {k: v for k, v in sample_input_dict_full.items() if k not in features_to_drop and k != 'Sektor'}
                
                # Panggil fungsi prediksi dari ml_credit_risk_predictor
                importlib.reload(ml_credit_risk_predictor)
                _load_res_output = ml_credit_risk_predictor._load_resources() # Panggil fungsi load secara eksplisit
                
                prediction_result = ml_credit_risk_predictor.predict_credit_risk_ml(
                    financial_data_dict=sample_input_dict_for_pred, 
                    sector=sample_input_series['Sektor']
                )

                sample_predictions.append({
                    'Perusahaan': sample_input_series['NamaPerusahaan'],
                    'Sektor_Asli': sample_input_series['Sektor'],
                    'Tahun_Asli': sample_input_series['PeriodeTahun'],
                    'Kategori_Risiko_Asli': sample_input_series[incremental_model_trainer.TARGET_COLUMN],
                    'Prediksi_Kategori_Risiko': prediction_result.get('risk_category'),
                    'Probabilitas': prediction_result.get('probabilities'),
                    'Error_Prediksi': prediction_result.get('error')
                })

        if sample_predictions:
            df_sample_predictions = pd.DataFrame(sample_predictions)
            print("\nContoh Hasil Prediksi pada Sampel Data:")
            display(df_sample_predictions)
        else:
            print("Tidak dapat membuat prediksi sampel.")
            
    elif not all_data.empty:
        print("Model atau preprocessor final tidak ditemukan. Pastikan pelatihan berhasil.")
else:
    print("Tidak ada data untuk melakukan prediksi.")


## 8. Kesimpulan

Notebook ini telah mendemonstrasikan:
1. Pemuatan dataset gabungan dari berbagai sektor.
2. Pelatihan model klasifikasi risiko kredit awal (`SGDClassifier`) pada subset data.
3. Pembaruan model secara inkremental menggunakan `partial_fit` ketika:
    - Data dari sektor baru ditambahkan.
    - Data historis baru untuk sektor yang sudah ada ditambahkan.
4. Penggunaan model yang telah dilatih/diperbarui untuk melakukan prediksi pada data baru.

Pendekatan ini memungkinkan model untuk beradaptasi dengan informasi baru dan pola dari berbagai sektor tanpa perlu melatih ulang keseluruhan model dari awal setiap kali ada data baru, yang penting untuk skenario di mana data datang secara bertahap dan beragam.